- http://www.cse.chalmers.se/~richajo/nlp2019/l3/Skip-gram%20with%20negative%20sampling.html
- https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf

# Prepare Dataset

In [3]:
from datasets import load_dataset

In [18]:
dataset = load_dataset(
  'wikitext', # データセット識別子
  'wikitext-103-v1') # データセット内のインスタンス名
# dataset.set_format(type='python') 
text = dataset['test']['text']

Reusing dataset wikitext (/Users/shotaimazeki/.cache/huggingface/datasets/wikitext/wikitext-103-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [67]:
text[:10]

['',
 ' = Robert Boulter = \n',
 '',
 ' Robert Boulter is an English film , television and theatre actor . He had a guest @-@ starring role on the television series The Bill in 2000 . This was followed by a starring role in the play Herons written by Simon Stephens , which was performed in 2001 at the Royal Court Theatre . He had a guest role in the television series Judge John Deed in 2002 . In 2004 Boulter landed a role as " Craig " in the episode " Teddy \'s Story " of the television series The Long Firm ; he starred alongside actors Mark Strong and Derek Jacobi . He was cast in the 2005 theatre productions of the Philip Ridley play Mercury Fur , which was performed at the Drum Theatre in Plymouth and the <unk> Chocolate Factory in London . He was directed by John Tiffany and starred alongside Ben Whishaw , Shane Zaza , Harry Kent , Fraser Ayres , Sophie Stanton and Dominic Hall . \n',
 ' In 2006 , Boulter starred alongside Whishaw in the play Citizenship written by Mark Ravenhill .

# Import

In [22]:
import torch
import torch.nn as nn

import numpy as np

import sys, time, os
from collections import Counter

In [100]:
params = {
    'device': 'cuda', # Device

    'n-neg-samples': 5, # Number of negative samples per positive sample
    'emb-dim': 64, # Embedding dimensionality

    'n-epochs': 1, # Number of epochs
    'max-words': 50000000, # How many words to consider in one epoch

    'batch-size': 1<<20, # Number of positive training instances in one batch
    'context-width': 5, # Maximal possible context width
    'prune-threshold': 1e-3, # Pruning threshold (see Mikolov's paper)
    'voc-size': 100000, # Maximal vocabulary size
    'ns-table-file': 'ns_table.txt', # Where to store the negative sampling table
    'ns-table-size': 1<<24, # Size of negative sampling table
    'ns-exp': 0.75, # Smoothing parameter for negative sampling distribution (see paper)
    'unknown-str': '<UNKNOWN>', # Dummy token for low-frequency words
    'lowercase': True, # Whether to lowercase the text
    'optimizer': 'adam', # Which gradient descent optimizer to use
    'lr': 1e-1, # Learning rate for the  optimizer

    # The test words for which we print the nearest neighbors periodically
    'testwords': ['apple', 'terrible', 'sweden', '1979', 'write'],
    # Number of nearest neighbors
    'n-testwords-neighbors': 5,
}

In [26]:
if params['device'] == 'cuda' and torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    print('Running on CUDA device.')
else:
    torch.set_default_tensor_type(torch.FloatTensor)
    print('Running on CPU.')

Running on CPU.


# building the vocabulary and negative sampling table

In [49]:
freqs = Counter()

for i, line in enumerate(text, 1):
    if params['lowercase']:
        line = line.lower()
    freqs.update(line.split()) #各単語の出現頻度

# 出現頻度が高い順にsort
freqs_sorted = sorted(freqs.items(),
                          key=lambda p: (p[1], p[0]),
                          reverse=True)

if len(freqs_sorted) > params['voc-size']-1:
    # vocab size以降の出現数のsum
    sum_freq_pruned = sum(f for _, f in freqs_sorted[params['voc-size']-1:])
else:
    sum_freq_pruned = 1
    
# 学習単語にないものをUNKトークンとしている
freqs_sorted = [(params['unknown-str'], sum_freq_pruned)] + freqs_sorted[:params['voc-size']-1]

ns_table = {}
sum_freq = 0
for w, freq in freqs_sorted:
    ns_freq = freq ** params['ns-exp'] # Smoothing parameter for negative sampling distribution (see paper)
    ns_table[w] = ns_freq
    sum_freq += ns_freq

scaler = params['ns-table-size'] / sum_freq
# word, 出現頻度, 出現頻度*scalerの整数値
ns_table = [(w, freq, int(round(ns_table[w]*scaler))) for w, freq in freqs_sorted]

In [92]:
class SGNSContextGenerator:

    def __init__(self, ns_table, params):
        self.corpus = text
        self.voc = { w:i for i, (w, _, _ ) in enumerate(ns_table) } # word2id
        self.batch_size = params['batch-size']
        self.ctx_width = params['context-width']
        self.lowercase = params['lowercase']
        self.word_count = 0

        # We define the pruning probabilities for each word as in Mikolov's paper.
        total_freq = sum(f for _, f, _ in ns_table)
        self.prune_probs = {}
        for w, f, _ in ns_table:
            self.prune_probs[w] = 1 - np.sqrt(params['prune-threshold'] * total_freq / f) # 出現頻度fが高いほど、右の値が小さくなり、probsが高くなる

    def prune(self, tokens):
        ps = np.random.random(size=len(tokens))
        # Remove some words from the input with probabilities defined by their frequencies.
        return [ w for w, p in zip(tokens, ps) if p >= self.prune_probs.get(w, 0) ] # get(w, 0) keyがない時に0を変えす。頻出の単語ほどpruneされやすい？？

    def batches(self):
        widths = np.random.randint(1, self.ctx_width+1, size=self.batch_size)
        width_ix = 0
        self.word_count = 0
        out_t = []
        out_c = []
        
        for line in text:
            if self.lowercase:
                line = line.lower()
            tokens = line.split()
            self.word_count += len(tokens)
            # 各単語を確率に基づいて削除。ns_tableにない単語はUNKトークンとする。
            encoded = [ self.voc.get(t, 0) for t in self.prune(tokens) ]
            
            for i, t in enumerate(encoded):
                # width_ixじゃなくてi??
                w = widths[width_ix]
                width_ix += 1

                # skipgram用のtarget--context ペアを生成。形は少し歪な気もするがtargetとcontextのペア。
                start = max(0, i-w)
                end = min(i+w+1, len(encoded))
                for j in range(start, end):
                    if j != i:
                        out_t.append(encoded[i])
                        out_c.append(encoded[j])
                        if len(out_t) == self.batch_size:
                            yield out_t, out_c
                            widths = np.random.randint(1, self.ctx_width+1, size=self.batch_size)
                            width_ix = 0
                            out_t = []
                            out_c = []
                    
            if len(out_t) > 0:
                yield out_t, out_c


ctx_gen = SGNSContextGenerator(ns_table, params)

In [94]:
class SGNSModel(nn.Module):

    def __init__(self, voc, params):
        super().__init__()
        
        voc_size = len(voc)
        self.w = nn.Embedding(voc_size, params['emb-dim'])
        self.c = nn.Embedding(voc_size, params['emb-dim'])
        self.voc = voc # word2id
        self.ivoc = { i:w for w, i in voc.items() } # id2word

    def forward(self, tgt, ctx):       
        # tgt is a 1-dimensional tensor containing target word ids
        # ctx is a 2-dimensional tensor containing positive and negative context ids for each target
        tgt_emb = self.w(tgt)
        n_batch, emb_dim = tgt_emb.shape
        tgt_emb = tgt_emb.view(n_batch, 1, emb_dim)
        
        n_ctx = ctx.shape[1] # negative context
        ctx_emb = self.c(ctx)
        ctx_emb = ctx_emb.transpose(1, 2)
        dots = tgt_emb.bmm(ctx_emb) # 行列積
        
        dots = dots.view(n_batch, n_ctx)
        return dots
    
    def nearest_neighbors(self, words, n_neighbors):
        words_ix = torch.as_tensor([self.voc[w] for w in words]) # word2id
        voc_size, emb_dim = self.w.weight.shape
        test_emb = self.w(words_ix).view(len(words), 1, emb_dim)
        all_emb = self.w.weight.view(1, voc_size, emb_dim)

        # wordsと全単語の類似度を計算
        sim_func = nn.CosineSimilarity(dim=2)
        scores = sim_func(test_emb, all_emb)
        near_nbr = scores.topk(n_neighbors+1, dim=1)
        values = near_nbr.values[:,1:]
        indices = near_nbr.indices[:, 1:]
        
        out = []
        for ixs, vals in zip(indices, values):
            out.append([ (self.ivoc[ix.item()], val.item()) for ix, val in zip(ixs, vals) ])
        return out
        
    
    def cosine_similarity(self, word1, word2):    
        # 不要か？？
        # We just look up the two embeddings and use PyTorch's built-in cosine similarity.
        v1 = self.w(torch.as_tensor(self.voc[word1]))
        v2 = self.w(torch.as_tensor(self.voc[word2]))
        sim = nn.CosineSimilarity(dim=0)
        return sim(v1, v2).item()
    
model = SGNSModel(ctx_gen.voc, params)

torch.Size([10, 1])

In [112]:
y_pos = torch.ones((10, 1))
y_neg = torch.zeros((10, 100))
torch.cat([y_pos, y_neg], dim=1)

tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.]])

In [101]:
class SGNSTrainer:

    def __init__(self, instance_gen, model, ns_table, params):
        self.instance_gen = instance_gen
        self.model = model
        self.n_epochs = params['n-epochs']
        self.max_words = params.get('max-words')
        n_batch = params['batch-size']
        self.n_ns = params['n-neg-samples']

        if params['optimizer'] == 'adam':
            self.optimizer = torch.optim.Adam(self.model.parameters(), lr=params['lr'])
        elif params['optimizer'] == 'sgd':
            self.optimizer = torch.optim.SGD(self.model.parameters(), lr=params['lr'])
        self.loss = nn.BCEWithLogitsLoss()

        ns_table_expanded = []
        for i, (_, _, count) in enumerate(ns_table):
            # 出現頻度が高いほど、samplingされやすくなる。
            ns_table_expanded.extend([i] * count)
        self.ns_table = torch.as_tensor(ns_table_expanded)
        
        y_pos = torch.ones((n_batch, 1))
        y_neg = torch.zeros((n_batch, self.n_ns))
        self.y = torch.cat([y_pos, y_neg], dim=1) # shape: [n_batch, 1+n_ns], 左端の列のみ1, その他は0, 

        self.testwords = params['testwords']
        self.n_testwords_neighbors = params['n-testwords-neighbors']

        self.epoch = 0
        
    def print_test_nearest_neighbors(self):
        nn_lists = self.model.nearest_neighbors(self.testwords, self.n_testwords_neighbors)
        for w, nn_list in zip(self.testwords, nn_lists):
            print(w, end=':\n')
            for nn, sim in nn_list:
                print(f' {nn} ({sim:.3f})', end='')
            print()
        
        print('------------------------------------')
        
    def make_negative_sample(self, batch_size):
        neg_sample_ixs = torch.randint(len(self.ns_table), (batch_size, self.n_ns)) # len()の値以下の数値をランダムに、第二引数のshapeで生成
        return self.ns_table.take(neg_sample_ixs) # negative sampling
            
    def train(self):

        print_interval = 5000000
        
        while self.epoch < self.n_epochs:
            print(f'Epoch {self.epoch+1}.')

            # For diagnostics.
            n_pairs = 0
            sum_loss = 0
            total_pairs = 0
            n_batches = 0
            t0 = time.time()
            
            # 各lineのtarget--context ペアを生成
            for t, c_pos in self.instance_gen.batches():

                batch_size = len(t)
                
                t = torch.as_tensor(t)                
                c_pos = torch.as_tensor(c_pos)
                c_pos = c_pos.view(batch_size, 1) # 正例, batch_sizeごとに並び替えている
                c_neg = self.make_negative_sample(batch_size) # 負例
                c = torch.cat([c_pos, c_neg], dim=1)
                self.optimizer.zero_grad()
                scores = self.model(t, c)
                loss = self.loss(scores, self.y[:batch_size])
                loss.backward()
                self.optimizer.step()

                # We'll print some diagnostics periodically.
                sum_loss += loss.item()
                n_pairs += batch_size
                n_batches += 1
                if n_pairs > print_interval:
                    total_words = self.instance_gen.word_count
                    total_pairs += n_pairs
                    t1 = time.time()                    
                    print(f'Pairs: {total_pairs}, words: {total_words}, loss: {sum_loss / n_batches:.4f}, time: {t1-t0:.2f}')
                    self.print_test_nearest_neighbors()
                    if self.max_words and total_words > self.max_words:
                        break
                    n_pairs = 0
                    sum_loss = 0
                    n_batches = 0
                    t0 = time.time()
                    
            self.epoch += 1

trainer = SGNSTrainer(ctx_gen, model, ns_table, params)
trainer.train()

Epoch 1.
Pairs: 5038226, words: 12543, loss: 0.2013, time: 10.49
apple:
 tassels (0.505) occasionally (0.471) siltstone (0.434) fray (0.425) participate (0.420)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) bradley (0.453) implementation (0.438)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) reality (0.395) 82 (0.389)
1979:
 colonel (0.478) contrasting (0.457) caught (0.450) d (0.443) detonations (0.415)
write:
 day (0.447) reflections (0.431) armoured (0.429) net (0.427) floatplane (0.423)
------------------------------------
Pairs: 10089836, words: 19088, loss: 0.1844, time: 10.12
apple:
 tassels (0.505) occasionally (0.471) mass (0.439) siltstone (0.434) fray (0.425)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) bradley (0.453) implementation (0.438)
sweden:
 goldman (0.443) eleventh (0.405) crassus (0.396) plácido (0.395) reality (0.395)
1979:
 colonel (0.478) contrasting (0.457) d (0.443) resistance (0.413) offs (0.410)
write:
 armoured (0.462) reflec

Pairs: 86164029, words: 51784, loss: 0.1562, time: 8.97
apple:
 tassels (0.505) siltstone (0.434) theirs (0.427) fray (0.425) petyarre (0.408)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) implementation (0.438) aw (0.424)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) 82 (0.389) yokoi (0.386)
1979:
 kenyon (0.686) selma (0.624) issue (0.602) wolverine (0.593) appeared (0.583)
write:
 telescopic (0.623) 1923 (0.614) politically (0.603) almirante (0.597) communion (0.596)
------------------------------------
Pairs: 91258559, words: 51839, loss: 0.1427, time: 7.42
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) theirs (0.412) petyarre (0.408)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) implementation (0.438) aw (0.424)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) 82 (0.389) yokoi (0.386)
1979:
 kenyon (0.709) selma (0.626) appeared (0.609) issue (0.604) wolverine (0.593)
write:
 telescopic (0.618) communion (0.604) tabular (0.595) politi

Pairs: 168443042, words: 67310, loss: 0.1592, time: 9.49
apple:
 tassels (0.505) siltstone (0.434) lower (0.431) fray (0.425) planned (0.416)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) implementation (0.438) aw (0.424)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 kenyon (0.770) faber (0.617) yamato (0.614) granta (0.609) measurements (0.597)
write:
 flanks (0.607) beastie (0.607) claws (0.604) almirante (0.602) sight (0.600)
------------------------------------
Pairs: 173485078, words: 68084, loss: 0.1592, time: 9.26
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) lower (0.414) planned (0.412)
terrible:
 sum (0.514) 294 (0.465) undertakings (0.454) implementation (0.438) aw (0.424)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 kenyon (0.742) yamato (0.614) faber (0.601) measurements (0.601) granta (0.599)
write:
 claws (0.608) flanks (0.605) salvador (0.604) alloy (0.602) 

Pairs: 250953287, words: 84968, loss: 0.1741, time: 10.32
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) castles (0.396) mcmahon (0.395)
terrible:
 sealing (0.675) playoff (0.641) enfant (0.600) deadly (0.589) lagged (0.581)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 invaded (0.744) kenyon (0.634) iraq (0.619) negotiated (0.615) generated (0.574)
write:
 hermes (0.707) modernization (0.621) acute (0.620) communion (0.619) alloy (0.614)
------------------------------------
Pairs: 255957211, words: 85351, loss: 0.1709, time: 8.64
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) turret (0.400) castles (0.396)
terrible:
 sealing (0.688) playoff (0.630) enfant (0.597) deadly (0.585) lagged (0.578)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 invaded (0.737) kenyon (0.624) iraq (0.622) negotiated (0.609) generated (0.584)
write:
 hermes (0.700) acute (0.629) communion (0.621) moder

Pairs: 334173727, words: 97414, loss: 0.1742, time: 10.82
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) castles (0.396) mcmahon (0.395)
terrible:
 sealing (0.689) playoff (0.638) enfant (0.628) establishment (0.568) civilians (0.567)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 kenyon (0.713) invaded (0.706) iraq (0.619) negotiated (0.590) generated (0.570)
write:
 hermes (0.697) modernization (0.621) simplify (0.614) purgation (0.613) communion (0.611)
------------------------------------
Pairs: 339532132, words: 97699, loss: 0.1722, time: 10.81
apple:
 tassels (0.505) siltstone (0.434) fray (0.425) turret (0.396) castles (0.396)
terrible:
 sealing (0.694) enfant (0.630) playoff (0.626) civilians (0.574) leaves (0.569)
sweden:
 goldman (0.443) crassus (0.396) plácido (0.395) yokoi (0.386) parade (0.374)
1979:
 kenyon (0.714) invaded (0.695) iraq (0.615) negotiated (0.585) thatched (0.576)
write:
 hermes (0.680) modernization (0.

...

Pairs: 1770500299, words: 223014, loss: 0.2212, time: 11.24
apple:
 maps (0.748) bowed (0.667) ios (0.644) pathé (0.627) inc (0.621)
terrible:
 enfant (0.684) sealing (0.607) deprived (0.582) antagonisms (0.574) nez (0.569)
sweden:
 1998 (0.405) crassus (0.396) executioner (0.370) barely (0.368) indies (0.361)
1979:
 christopher (0.578) gangsters (0.572) summoned (0.568) vague (0.565) alters (0.551)
write:
 craved (0.669) ill (0.632) 339 (0.625) transmitter (0.585) spec (0.584)
------------------------------------
Pairs: 1775724933, words: 223398, loss: 0.2229, time: 11.11
apple:
 maps (0.752) bowed (0.673) ios (0.641) inc (0.631) pathé (0.630)
terrible:
 enfant (0.687) sealing (0.604) deprived (0.581) antagonisms (0.576) nez (0.572)
sweden:
 1998 (0.417) crassus (0.396) empire (0.380) barely (0.370) executioner (0.370)
1979:
 christopher (0.585) gangsters (0.578) summoned (0.574) vague (0.570) hippodrome (0.560)
write:
 craved (0.671) ill (0.631) 339 (0.629) transmitter (0.588) spec (

Pairs: 1854692622, words: 227336, loss: 0.2188, time: 11.43
apple:
 maps (0.754) bowed (0.682) ios (0.666) google (0.662) inc (0.654)
terrible:
 enfant (0.687) sealing (0.615) imitating (0.575) antagonisms (0.569) uniform (0.569)
sweden:
 denmark (0.723) luxembourg (0.695) norway (0.620) netherlands (0.608) slovakia (0.598)
1979:
 gangsters (0.619) christopher (0.566) vague (0.566) summoned (0.564) kuwait (0.541)
write:
 craved (0.646) 339 (0.634) ill (0.594) transmitter (0.591) spec (0.580)
------------------------------------
Pairs: 1860008662, words: 227415, loss: 0.2182, time: 10.76
apple:
 maps (0.759) bowed (0.680) ios (0.667) google (0.665) inc (0.654)
terrible:
 enfant (0.685) sealing (0.613) imitating (0.573) antagonisms (0.568) 204 (0.567)
sweden:
 denmark (0.724) luxembourg (0.698) norway (0.633) netherlands (0.619) slovakia (0.602)
1979:
 gangsters (0.614) summoned (0.573) vague (0.572) christopher (0.570) kuwait (0.551)
write:
 craved (0.655) 339 (0.629) spec (0.593) kroeg

Pairs: 1940159751, words: 230577, loss: 0.2207, time: 11.55
apple:
 maps (0.758) ios (0.691) bowed (0.674) google (0.659) backlot (0.614)
terrible:
 enfant (0.673) sealing (0.633) uniform (0.607) aesthetic (0.583) gusted (0.561)
sweden:
 denmark (0.726) luxembourg (0.725) norway (0.687) netherlands (0.646) finland (0.615)
1979:
 christopher (0.621) gangsters (0.613) summoned (0.574) operates (0.565) morales (0.559)
write:
 craved (0.674) 339 (0.635) ill (0.612) spec (0.605) fascination (0.583)
------------------------------------
Pairs: 1945551181, words: 230744, loss: 0.2210, time: 11.23
apple:
 maps (0.754) ios (0.693) bowed (0.673) google (0.658) backlot (0.612)
terrible:
 enfant (0.674) sealing (0.640) uniform (0.600) aesthetic (0.592) exemplifies (0.565)
sweden:
 denmark (0.726) luxembourg (0.726) norway (0.690) netherlands (0.650) finland (0.615)
1979:
 christopher (0.615) gangsters (0.600) summoned (0.562) operates (0.562) mobilizing (0.555)
write:
 craved (0.666) 339 (0.622) il

Pairs: 2027258348, words: 235021, loss: 0.2239, time: 11.68
apple:
 maps (0.751) ios (0.704) google (0.666) bowed (0.666) inc (0.652)
terrible:
 enfant (0.659) sealing (0.625) aesthetic (0.592) uniform (0.590) exemplifies (0.575)
sweden:
 luxembourg (0.767) denmark (0.756) norway (0.714) slovakia (0.665) netherlands (0.648)
1979:
 gangsters (0.597) christopher (0.585) operates (0.576) kuwait (0.566) vague (0.559)
write:
 craved (0.655) 339 (0.651) outlook (0.602) spec (0.598) fascination (0.597)
------------------------------------
Pairs: 2032752433, words: 235389, loss: 0.2227, time: 11.73
apple:
 maps (0.754) ios (0.715) google (0.666) bowed (0.664) inc (0.655)
terrible:
 enfant (0.655) sealing (0.618) aesthetic (0.591) uniform (0.583) exemplifies (0.573)
sweden:
 luxembourg (0.771) denmark (0.757) norway (0.724) slovakia (0.658) denver (0.646)
1979:
 gangsters (0.594) christopher (0.585) operates (0.576) kuwait (0.565) exit (0.561)
write:
 craved (0.658) 339 (0.656) outlook (0.614) 

Pairs: 2116144944, words: 240649, loss: 0.2264, time: 11.99
apple:
 maps (0.731) ios (0.729) google (0.679) inc (0.670) bowed (0.661)
terrible:
 enfant (0.635) releasing (0.584) courtyard (0.559) imitating (0.559) sealing (0.558)
sweden:
 luxembourg (0.770) denmark (0.749) norway (0.714) slovakia (0.657) netherlands (0.654)
1979:
 vague (0.609) gangsters (0.605) christopher (0.592) mobilizing (0.576) invaded (0.569)
write:
 craved (0.658) 339 (0.642) hyde (0.611) remarkable (0.606) transmitter (0.603)
------------------------------------
Pairs: 2121765498, words: 240699, loss: 0.2241, time: 11.85
apple:
 maps (0.733) ios (0.727) google (0.678) inc (0.670) bowed (0.658)
terrible:
 enfant (0.649) releasing (0.587) sealing (0.582) courtyard (0.547) josepha (0.544)
sweden:
 luxembourg (0.771) denmark (0.749) norway (0.715) slovakia (0.658) netherlands (0.650)
1979:
 vague (0.619) gangsters (0.608) christopher (0.596) mobilizing (0.584) invaded (0.576)
write:
 craved (0.649) 339 (0.640) spe